# Noms du binome: OUMOUHOU, CHADLI

# Prénoms du binome: Hicham, Mohamed amine

## Note: exporter le compte rendu basé sur le notebook au format pdf


# Entropie et génération de mots de passe
* L'objectif de ce tp est de continuer à se familiariser avec la notion d'entropie, mais aussi de comprendre le lien qu'il existe entre cette mesure informationelle et la sécurité d'un générateur (humain ou executable) de mots de passes
* Ainsi, nous proposons d'étudier l'entropie d'un tel générateur, et ce en fonction du modèle probabiliste considéré pour le modéliser (contruit à partir d'une lettre, de deux lettres, de 4 lettres, ...). A l'aide de tirrages aléatoires, nous estimerons également le temps moyen nécessaire pour trouver un mot de passe à partir de ce modèle.
* A la fin de ce TP, nous considérerons un générateur de mots de passe spécifique qui générera un mot de passe en prennant **4 lettres consécutives dans un texte** (sans se soucier des espaces). Ces lettres peuvent faire parti d'un ou de plusieurs mots consécutifs.
* Nous faisons l'hypothèse que le texte n'est composé que des 26 lettres de l'alphabet, sans majuscules ni accents

Nous chercherons aussi à comprendre (voir dernière question):
- les bonnes pratiques pour le défenseur, i.e. la personne cherchant à générer/construire un système de génération de mots de passe.
- les bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe.

**Il est important de commenter vos réponses, en utilisant des cellules markdown**


In [84]:
import numpy as np

from numpy import genfromtxt
from pandas import read_csv
import pandas as pd
import time

#### Modèle monogramme (une lettre) : le générateur génère des mots de passe à partir des occurences des monogrammes
* On récupére des données composées de [lettre,frequence d'apparition de la lettre] (voir fichier csv pour [comma-separated-value](https://en.wikipedia.org/wiki/Comma-separated_values)) 

In [114]:
monogramme = read_csv('monogramme.csv')
freq_mono = (monogramme['frequency']).values
letters_mono = (monogramme['letters']).values

Q: Quelles sont les 5 lettres les plus représentées ?

In [115]:
lettres_plus_frequentes = letters_mono[freq_mono.argsort()[-5:][::-1]]
print("les plus fréquentes lettres sont:",lettres_plus_frequentes)

les plus fréquentes lettres sont: ['E' 'S' 'A' 'N' 'T']


Ecrire une fonction qui calcule l'entropie à partir d'un vecteur constitué de probabilités empiriques (note, il est important de bien *gérer* le cas ou la probabilité est nulle).

In [116]:
def entropie(freq):
    ent = 0
    for i in range(len(freq)):
        if freq[i] != 0:
            ent += freq[i]*np.log2(freq[i])
    return -ent

exemple_freq = np.array([0.1,0.2,0.3,0.4,0])
print("entropie de exemple_freq:",entropie(exemple_freq))

entropie de exemple_freq: 1.8464393446710154


Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

__réponse__ : On peut voir les chiffres de mot de passe comme des variables aléatoires indépendantes (approche naive), donc l'entropie 
de mot de passe est la somme des entropies de chaque chiffre.

In [117]:
entropie_mono_8 = 8 * entropie(freq_mono)
print("entropie de monogramme pour un mot de passe à 8 chiffres est:",entropie_mono_8.round(2),"bits")

entropie de monogramme pour un mot de passe à 8 chiffres est: 31.68 bits


Q: A l'aide de la fonction `np.random.choice()`, estimer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ? (note: ici le tirage n'est pas forcemment réaliste, car aléatoire, mais l'idée est surtout de mesurer le temps minimal nécessaire pour générer N mots de passes).

In [118]:
t = time.time()
List_mot_de_passe = [np.random.choice(letters_mono, 8, p = freq_mono) for i in range(100000)]
t_100000 = time.time() - t
print("le temps de génération de 100000 mots de passe est:",t_100000,"secondes")

le temps de génération de 100000 mots de passe est: 2.56378436088562 secondes


Nous definissons l'"entropie du devin" G (guessing entropie) comme le **nombre moyen d'essais successif nécessaires pour trouver un mot de passe à partir de notre générateur**.
    On peut montrer que $G\geq 2^H/4+1$ où $H$ est l'entropie de la source (voir le papier Password_Entropy_and_Password_Quality.pdf )

Q: calculer le minorant de $G$ pour ce modèle

In [119]:
min_G = 1 + (2**entropie_mono_8)/4
print("le nombre moyen minimal de mots de passe à tester pour trouver le bon est:",min_G.round(2),"mots de passe")

le nombre moyen minimal de mots de passe à tester pour trouver le bon est: 857904864.68 mots de passe


Q: combien de temps cela prendra-t-il pour trouver un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [120]:
temps_pour_génération = t_100000/100000
temps_trouver_mot_de_passe = min_G* temps_pour_génération
print("le temps moyen minimal pour trouver le bon mot de passe est:",(temps_trouver_mot_de_passe/60).round(2),"minutes")

le temps moyen minimal pour trouver le bon mot de passe est: 366.58 minutes


On propose maintenant d'utiliser un modèle plus évolué qui est construit à partir de la probabilité conjointe de deux lettres successives (bigramme)

In [121]:
bigramme = read_csv('bigramme.csv',keep_default_na=False)

#print(bigramme.head(5))
freq_bi = (bigramme['frequency']).values

letters_bi = (bigramme['letters']).values

bigramme.head(5)

,letters,frequency
0,AA,0.000252
1,AB,0.001760
2,AC,0.003482
3,AD,0.001960
4,AE,0.000200


Q: Quelles sont les 5 couples de lettres les plus représentés ?

In [122]:
lettres_plus_frequentes = letters_bi[freq_bi.argsort()[-5:][::-1]]
print("les plus fréquentes couple de lettres sont:",lettres_plus_frequentes)

les plus fréquentes couple de lettres sont: ['ES' 'EN' 'DE' 'LE' 'NT']


Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [123]:
entropie_bi_8 = 4 * entropie(freq_bi)
print("entropie de bigramme:",entropie_bi_8.round(2),"bits")

entropie de bigramme: 30.14 bits


Q: Pourquoi cette entropie est-elle inférieure à celle du modèle construit sur des monogrammes ? Quelle propriété théorique de l'entropie peut justifier ce constat  ?

- L'entropie de bigramme est inférieure à celle du monogramme car en prenant en compte les couples de lettres, on voit plus les lettres successives comme des variables indépendants et ainsi on réduit l'ensemble des possibilités.
- La proprieté théorique qui justifie ce constat est la suivante : l'entropie d'un ensemble de variables aléatoires est inférieure ou égale à la somme des entropies de ces variables aléatoires. En fait l'entropie d'un couple de lettres est inférieure à la somme des entropies de ces lettres. 
- __remarque__: la différence entre les deux entropies est faible, cela  signifier que l'inforamtion mutuelle entre les lettres est faible.

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [124]:
t = time.time()

List_mot_de_passe = [np.random.choice(letters_bi, 4, p = freq_bi) for i in range(100000)]
t_100000 = time.time() - t

print("le temps de génération de 100000 mots de passe à 8 chiffre avec le bigramme:",t_100000,"secondes")


le temps de génération de 100000 mots de passe à 8 chiffre avec le bigramme: 2.9846417903900146 secondes


__réponse__ : On voit que le temps nécessaire pour tirer 100 000 mots de passe est plus grand que celui du monogramme, car dans le cas du bigramme on plus de possibilités pour la fonction `np.random.choice()`.

Q: calculer le minorant de $G$ pour ce modèle

In [125]:
min_G = 1 + (2**entropie_bi_8)/4
print("le nombre moyen minimal de mots de passe à tester pour trouver le bon est:",min_G.round(2),"mots de passe")

le nombre moyen minimal de mots de passe à tester pour trouver le bon est: 296254956.7 mots de passe


__Remarque__ : On remarque quand meme qu'on a réduit considérablement le nombre de mot de passe à tester en utilisant le bigramme.

Q: combien de temps cela prendra-t-il pour trouver un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [127]:
temps_pour_génération = t_100000/100000
temps_trouver_mot_de_passe = min_G* temps_pour_génération
print("temps pour trouver un mot de passe en minute:",temps_trouver_mot_de_passe/60,"min")

temps pour trouver un mot de passe en minute: 147.36915406360063 min


__Remarque__ : On a aussi réduit le temps de recherche du mot de passe preque de moitié.

Q: **Modèle Uniforme:** si maintenant on change de stratégie et on tire aléatoirement chaque lettre de l'alphabet de façon uniforme, quelle est l'entropie de ce nouveau générateur ?

In [130]:
uniform_entropie = 8 * np.log2(26)
print("Entropie de modèle uniforme est:",uniform_entropie.round(2),"bits")

Entropie de modèle uniforme est: 37.6 bits


__Remarque__ : l'entropie est plus grande que celle du monogramme et du bigramme, car dans ce cas on néglige la fréquence d'apparition des lettres.

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [132]:
t = time.time()
list_mot_de_passe = [np.random.choice(letters_mono, 8) for i in range(100000)]
t_100000 = time.time() - t
print("le temps de génération de 100000 mots de passe est:",t_100000,"secondes")

le temps de génération de 100000 mots de passe est: 1.7060933113098145 secondes


__Remarque__ : le temps nécessaire pour tirer 100 000 mots de passe est plus petit que celui du monogramme et du bigramme, car dans ce cas tous les lettres sont équiprobables.

Q: calculer le minorant de $G$ pour ce modèle

In [133]:
min_G = 1 + (2**uniform_entropie)/4
print("le nombre moyen minimal de mots de passe à tester pour trouver le bon est:",min_G.round(2),"mots de passe")

le nombre moyen minimal de mots de passe à tester pour trouver le bon est: 52206766145.0 mots de passe


__Remarque__ : Le minorant de G est plus grand que celui du monogramme et du bigramme, car on a une entropie plus grande.

Q: dans ce cas précis, quelle est la valeur exacte de $G$?

__Réponse__ : Vu qu'on a $26^8$ possibilités, on a $G=26^8/2$.

In [135]:
G = (26**8)/2
print("le nombre de mots de passe moyen à tester pour trouver le bon est:",G,"mots de passe")  

le nombre de mots de passe moyen à tester pour trouver le bon est: 104413532288.0 mots de passe


Q: combien de temps cela prendra-t-il pour trouver un mot de passe en utilisant le générateur codé précédemment ? (en minutes)

In [136]:
temps_pour_génération = t_100000/100000
temps_trouver_mot_de_passe = G* temps_pour_génération
print("temps pour trouver un mot de passe en minute:",temps_trouver_mot_de_passe/60,"minutes")

temps pour trouver un mot de passe en minute: 29689.871507798023 minutes


__Remarque__: Le temps de recherche est très grand par rapport aux autres modèles, car on a négligé la fréquence d'apparition des lettres et on a pris les lettres équiprobables. ce qui montre l'importance des modèles précédents.

Q: implémenter une attaque pratique qui consiste à:
1. **pour le défenseur: (la personne qui génère le mot de passe)** tirer un mot de passe de 4 lettres consécutives à partir de ce texte de Victor Hugo (texteFrancais.txt) tiré des Misérables.  
2. **pour l'attaquant:** utiliser le modèle bigramme pour générer des mots de passe et minimiser le nombre d'essais. Pour cela on pourra :
    * dans un premier temps pré-calculer un **dictionnaire**, qui contriendra un nombre de MdP générés classés dans l'ordre du plus probable au moins probable et qui ne contient pas de doublons 
    * dans un deuxième temps appeler ce dictionnaire pour comparer chacune de ses entrées au mot de passe généré.
3. Il faudra faire ses tests plusieurs fois afin de d'obtenir un nombre moyens d'appel au dictionnaire nécessaire
4. Il sera intéressant de comparer le nombre trouvé à la valeur de G (qui est une borne inférieure)
5. Question annexe: Par un simple calcul, si le générateur utilisé n'est plus ce générateur mais un générateur qui tire chaque lettre de façon équiprobable, rappeler la valeur de G. Comparer cette valeur avec la valeur trouvée en utilisant la stratégie "des 4 lettres consécutives".

In [138]:
## Fonction générant un mot de passe
def get_passwd():
    text_hugo = open("texteFrancais.txt","r")
    str_hugo = str(text_hugo.read())

    # On remplace des lettres avec accent avec des lettres sans accent
    str_hugo = str_hugo.replace("Â", "A")
    str_hugo = str_hugo.replace("Ù", "U")
    str_hugo = str_hugo.replace("Ô", "O")
    size_txt = len(str_hugo)

    idx_rand = np.random.randint(size_txt-4)
    #print(idx_rand)

    psswd = str_hugo[idx_rand:idx_rand+4]
    return(psswd)

In [139]:
get_passwd()

'EVIL'

In [140]:
# Génération du dictionnaire
df = pd.DataFrame()
tab_passwd = []
tab_proba = []
# Tri des fréquences d'apparition
sorted_bigramme  = bigramme.sort_values(by=['frequency'], ascending=False) # Pour renitialiser les indices de sorte que le premier élément aura l'indice 0
sorted_bigramme = sorted_bigramme.reset_index(drop=True) 

for i in range(len(sorted_bigramme)) :
    for j in range(len(sorted_bigramme)) :
        mdp = sorted_bigramme['letters'][i] + sorted_bigramme['letters'][j]
        proba = sorted_bigramme['frequency'][i] * sorted_bigramme['frequency'][j]
        tab_passwd.append(mdp)
        tab_proba.append(proba)
df["Password"] = tab_passwd
df['Prob'] = tab_proba

In [141]:
df = df.drop_duplicates(subset="Password")

In [142]:
df.head(5)

,Password,Prob
0,ESES,0.000567
1,ESEN,0.000506
2,ESDE,0.000466
3,ESLE,0.000449
4,ESNT,0.000405


__Remarque__ : Il sera intéressant de calculer l'entropie de ce générateur ainsi que le nombre d'essais moyen nécessaire pour trouver le mot de passe.

In [145]:
entropie_df = entropie(df['Prob'])
print("Entropie du dictionnaire est:",entropie_df.round(2),"bits")
G = 1+(2**entropie_df)/4
print("le nombre moyen minimal de mots de passe à tester pour trouver le bon est:",G.round(2),"mots de passe")

Entropie du dictionnaire est: 15.07 bits
le nombre moyen minimal de mots de passe à tester pour trouver le bon est: 8607.03 mots de passe


In [161]:
# Attaques sur 1000 mots de passes

nb_trial = 1000
# On stocke le nombre de tentatives pour trouver le mot de passe
vec_nb_trials = np.zeros(nb_trial)
# On stocke le mot de passe
tab_passwd = []
# On stocke la probabilité d'apparition du mot de passe
mdp_by_nbrT = pd.DataFrame()
for i in range(nb_trial):
    password = get_passwd()
    Number_tentations = 0
    for mdp in df["Password"] :
        Number_tentations += 1
        if(mdp == password) :
            break
    tab_passwd.append(mdp)
    vec_nb_trials[i] = Number_tentations

In [162]:
mdp_by_nbrT["Mot de passe"] = tab_passwd
mdp_by_nbrT["Nbr tentations pour trouver"] = vec_nb_trials

In [163]:
mdp_by_nbrT.head(5)

,Mot de passe,Nbr tentations pour trouver
0,SEIL,9485.0
1,NOUV,57600.0
2,SERE,9470.0
3,ASEN,37858.0
4,EMME,22336.0


In [164]:
mean = vec_nb_trials.mean()
print(f"Le nombre moyen pour deviner le mot de passe :{mean}")

Le nombre moyen pour deviner le mot de passe :40134.051


__Réponse__ : Le nombre d'essais moyen nécessaire pour trouver le mot de passe est effevtifement inférieur à la valeur de G.

## Conclusions 

Définir des bonnes pratiques pour le défenseur, i.e. la personne cherchant à concevoir un système de génération de mots de passe ? 

__Réponse__: 
-  Choisir les lettres de telle sorte à ce que qu'ils soient les plus indépendants possibles (inclure les lettres les moins fréquentes dans le texte).
-  Choisir le mots de passe le plus long possible.
-  Changer le mot de passe régulièrement.
-  Éviter les informations personnelles 


Définior des bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe ?

__Réponse__ :

- Utiliser un modèle probabiliste qui prend en compte les lettres successives (bigramme, trigramme, ...) pour diminuer l'entropie.
- Utiliser un dictionnaire de mots de passe générés à partir de texte eventuellement utilisé par le défenseur (Livre, article, information personnelle, ...).   


**To do**

## Un peu de lecture
Cet article montre comment des hackers, à partir de leaks de bases de mots de passes, peuvent rapidement arriver à trouver le votre:
https://arstechnica.com/information-technology/2013/05/how-crackers-make-minced-meat-out-of-your-passwords/


__Quelques points intéréssnts de l'article__: 
- Quelles sont quelques-unes des techniques couramment utilisées par les pirates informatiques pour déchiffrer les mots de passe hachés?

Une technique courante utilisée par les pirates informatiques consiste à craquer d'abord les mots de passe les plus faibles, surtout lorsque les hachages contiennent un sel cryptographique. Une autre technique consiste à utiliser des ensembles de règles personnalisés ciblant des hachages spécifiques, ce qui nécessite une connaissance du site d'où la liste de mots de passe a été extraite. De plus, les pirates informatiques peuvent recourir à des attaques par force brute, des attaques par dictionnaire ou des attaques hybrides pour craquer des mots de passe.

- Comment les particuliers et les entreprises peuvent-ils se protéger contre les attaques de craquage de mot de passe?

Les particuliers et les entreprises peuvent se protéger contre les attaques de craquage de mot de passe en utilisant des mots de passe forts et uniques pour chaque compte, en mettant en place l'authentification à deux facteurs et en changeant régulièrement les mots de passe . Il est également recommandé d'utiliser des gestionnaires de mots de passe pour générer des codes d'accès longs et aléatoires qui sont uniques pour chaque site, car ces types de mots de passe ne peuvent être craqués que par force brute et sont les plus difficiles à récupérer . De plus, les entreprises peuvent imposer des politiques de mot de passe strictes pour les employés et effectuer régulièrement des audits de mot de passe pour s'assurer que les politiques de mot de passe sont correctement appliquées .